R1CS to QAP
=============

In [1]:
%load_ext autoreload
%autoreload 2

## Example

In [15]:
# ref: https://medium.com/@VitalikButerin/quadratic-arithmetic-programs-from-zero-to-hero-f6d558cea649

from functools import partial
from klefki.zkp.r1cs import R1CS, mul
from klefki.zkp.qap import R1CS2QAP, proof, verify
from klefki.curves.barreto_naehrig.bn128 import BN128FP as F
import ast



In [16]:
# map int to field
ciphers = [1,2,3,4,5,6,7,8,9]
times = 5

@R1CS.r1cs(F)
def f(x, k, c):
    y = x + c + k
    return y ** 3

@R1CS.r1cs(F, globals())
def mimc(x, k):
    for i in range(times):
        c = ciphers[i]
        x = f(x, k, c)
    return x + k

In [17]:
f.flatcode

[['+', 'Sym::0', 'x', 'c'],
 ['+', 'y', 'Sym::0', 'k'],
 ['*', 'Sym::1', 'y', 'y'],
 ['*', '~out', 'Sym::1', 'y']]

In [18]:
mimc.flatcode

[['set', 'c', 1],
 ['+', 'Local<Rc(0)>Sym::0', 0, 2],
 ['+', 'Local<Rc(0)>y', 'Local<Rc(0)>Sym::0', 1],
 ['*', 'Local<Rc(0)>Sym::1', 'Local<Rc(0)>y', 'Local<Rc(0)>y'],
 ['*', 'x::0', 'Local<Rc(0)>Sym::1', 'Local<Rc(0)>y'],
 ['set', 'c::1', 2],
 ['+', 'Local<Rc(1)>Sym::0', 0, 2],
 ['+', 'Local<Rc(1)>y', 'Local<Rc(1)>Sym::0', 1],
 ['*', 'Local<Rc(1)>Sym::1', 'Local<Rc(1)>y', 'Local<Rc(1)>y'],
 ['*', 'x::2', 'Local<Rc(1)>Sym::1', 'Local<Rc(1)>y'],
 ['set', 'c::3', 3],
 ['+', 'Local<Rc(2)>Sym::0', 0, 2],
 ['+', 'Local<Rc(2)>y', 'Local<Rc(2)>Sym::0', 1],
 ['*', 'Local<Rc(2)>Sym::1', 'Local<Rc(2)>y', 'Local<Rc(2)>y'],
 ['*', 'x::4', 'Local<Rc(2)>Sym::1', 'Local<Rc(2)>y'],
 ['set', 'c::5', 4],
 ['+', 'Local<Rc(3)>Sym::0', 0, 2],
 ['+', 'Local<Rc(3)>y', 'Local<Rc(3)>Sym::0', 1],
 ['*', 'Local<Rc(3)>Sym::1', 'Local<Rc(3)>y', 'Local<Rc(3)>y'],
 ['*', 'x::6', 'Local<Rc(3)>Sym::1', 'Local<Rc(3)>y'],
 ['set', 'c::7', 5],
 ['+', 'Local<Rc(4)>Sym::0', 0, 2],
 ['+', 'Local<Rc(4)>y', 'Local<Rc(4)>Sym::

The format of a flatcode line is:

$$
\left[Op, Out, S_a, S_b\right]
$$

In [19]:
mimc.var

['~one',
 'x',
 'k',
 '~out',
 'c',
 'Local<Rc(0)>Sym::0',
 'Local<Rc(0)>y',
 'Local<Rc(0)>Sym::1',
 'x::0',
 'c::1',
 'Local<Rc(1)>Sym::0',
 'Local<Rc(1)>y',
 'Local<Rc(1)>Sym::1',
 'x::2',
 'c::3',
 'Local<Rc(2)>Sym::0',
 'Local<Rc(2)>y',
 'Local<Rc(2)>Sym::1',
 'x::4',
 'c::5',
 'Local<Rc(3)>Sym::0',
 'Local<Rc(3)>y',
 'Local<Rc(3)>Sym::1',
 'x::6',
 'c::7',
 'Local<Rc(4)>Sym::0',
 'Local<Rc(4)>y',
 'Local<Rc(4)>Sym::1',
 'x::8']

The format of variable is

$$
[One, Input_0, \cdots, Input_n, Output, S_0, S_1, \cdots, S_n]
$$


In [20]:
assert len(mimc.A[0]) == len(mimc.var)

For each line of flatcodes, we have $A_i.s \circ B_i.s == C_i.s$

In [21]:
s = mimc.witness(F(2))
s

[BN128FP::1,
 BN128FP::2,
 BN128FP::0,
 BN128FP::27,
 BN128FP::1,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27,
 BN128FP::2,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27,
 BN128FP::3,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27,
 BN128FP::4,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27,
 BN128FP::5,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27]

In [22]:
sum(mul(mimc.A[0], s)) * sum(mul(mimc.B[0], s)) == sum(mul(mimc.C[0], s))

True

## Gen QAP


In [23]:
r1cs = mimc.r1cs
r1cs

([[BN128FP::21888242871839275222246405745257275088696311157297823662689037894645226208582,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::1,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0],
  [BN128FP::2,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0],
  [BN128FP::1,
   BN128FP::0,
   BN128FP::0,
   BN12

In [24]:
QAP = R1CS2QAP(*r1cs, F(1234567890), field=F)
A, B, C, Z = QAP

In [25]:
s, H = proof(f.witness(3), A, B, C, Z, field=F)

In [26]:
verify(s, A, B, C, Z, H)

True

In [27]:
A, B, C, Z, H

([BN128FP::11756167876866112699184837963851660150222022753543126281397583854219100117835,
  BN128FP::0,
  BN128FP::11802133266539108046577272580813903544588502102169114593023195065891137430130,
  BN128FP::0,
  BN128FP::7632558597884491412763566817778515037029858892755138962445425647252549830447,
  BN128FP::215015020735758476994093100846584218473274177031446443307990825544860934448,
  BN128FP::2401619889399927381322034950163668466336227819361836046597617475687710550920,
  BN128FP::7824704598809110676093025659439897880715409414889204303227183838553528989695,
  BN128FP::0,
  BN128FP::21296943491041237097756191714460944382500289390354228947292466293154755687033,
  BN128FP::6807554577801142147974892897362748299621299790248351983801814979075987885367,
  BN128FP::8867970046576425819631466457410954599790358576604337124570870434032112026004,
  BN128FP::19567757027820532381636173075436303900117295464446278041788310033266216510087,
  BN128FP::0,
  BN128FP::1782356860066481989694677712160741999709